# Going Deeper 10. 도로 Segmentation

자율주행차량이 주행해야할 도로 영역을 찾는 도로 segmentation task 를 진행한다.  

U-Net 을 사용해서 이미지가 입력되면 도로의 영역을 segmenatation 하는 모델을 만들어보자.

<a href='https://ifh.cc/v-QlxZ0l' target='_blank'><img src='https://ifh.cc/g/QlxZ0l.gif' border='0'></a>